# General DA fittings and Fokker-Planck comparisons (Hénon map)

## Setup scripts if we are under SWAN

In [ ]:
# Working in the right path
%cd /eos/project/d/da-and-diffusion-studies/DA_Studies/Simulations/Models/loss_studies/notebooks

In [ ]:
# Install the libraries
import sys
!{sys.executable} -m pip install --user tqdm pynverse sixtrackwrap crank-nicolson-numba henon-map symplectic-map
!{sys.executable} -m pip install --user --upgrade sixtrackwrap 
!{sys.executable} -m pip install --user --upgrade crank-nicolson-numba 
!{sys.executable} -m pip install --user --upgrade henon-map 
!{sys.executable} -m pip install --user --upgrade symplectic-map
!export PYTHONPATH=$CERNBOX_HOME.local/lib/python3.7/site-packages:$PYTHONPATH

In [ ]:
# For this "presentation" only!
import warnings
warnings.filterwarnings('ignore')

## Imports

In [1]:
%matplotlib widget

In [1]:
# Base libraries
import math
import numpy as np
import scipy.integrate as integrate
from scipy.special import erf
import pickle
import itertools
from scipy.optimize import curve_fit

from numba import njit, prange

from tqdm.notebook import tqdm
import time
import matplotlib.pyplot as plt
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.ticker as ticker
from math import gcd

import pandas as pd

from scipy.special import lambertw
from scipy.interpolate import interp1d

import os

# Personal libraries
import sixtrackwrap as sx
import crank_nicolson_numba.nekhoroshev as nk

# Personal modules
import fit_utils as fit

/home/carlidel/Insync/carlo.montanari3@studio.unibo.it/OneDrive Biz/projects/loss_studies/notebooks/fit_utils.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Load data and setup original DA

In [2]:
savepath = "../data/"

sigma = 0.3
turn_samples = 100
min_turns = 10**3
max_turns = 10**6

# Do we want to load ALL the Hénon data files or should we skip some?
skipper = 1

cut_point = 1.0

In [3]:
epsilons = []

real_losses = []
gaussian_losses = []

real_DAs = []
gaussian_DAs = []

files = list(sorted(list(filter(lambda f: "henon_eps_" in f and "unif" not in f, os.listdir(savepath))), key=lambda f: float(f[10: -4])))[::skipper]

for f in tqdm(files):
    print("Loading file: ", f)
    #unif_f = "unif_" + f
    epsilon = float(f[10: -4])
    epsilons.append(epsilon)
    engine = sx.uniform_radial_scanner.load_values(savepath + f)
    #unif_engine = sx.uniform_scanner.load_values(savepath + unif_f)
    
    baseline_samples = engine.baseline_samples
    d_r = engine.dr
    turn_sampling = np.linspace(min_turns, max_turns, turn_samples, dtype=np.int)[::-1]
    
    print("Real part")
    real_DAs.append(engine.compute_DA_standard(turn_sampling))
    print("Done DA")
    
    print("Gaussian part")
    engine.assign_weights(sx.assign_symmetric_gaussian(sigma))
    #unif_engine.assign_weights(
    #    sx.assign_symmetric_gaussian(sigma, polar=False), radial_cut=cut_point)
    baseline = engine.compute_loss_cut(cut_point)
    
    gaussian_losses.append(engine.compute_loss(turn_sampling, cut_point, True))
    gaussian_DAs.append(
        fit.DA_from_symmetric_gaussian_loss(gaussian_losses[-1], sigma, cut_point))
    print("Done Gaussian")
    
    print("Gaussian loss but with 'Real' DA values")
    real_losses.append(fit.symmetric_gaussian_loss(real_DAs[-1], sigma, cut_point))
    print("Finished processing.")

Loading file:  henon_eps_1.pkl
Real part
Done DA
Gaussian part
Done Gaussian
Gaussian loss but with 'Real' DA values
Finished processing.
Loading file:  henon_eps_2.pkl
Real part
Done DA
Gaussian part
Done Gaussian
Gaussian loss but with 'Real' DA values
Finished processing.
Loading file:  henon_eps_3.pkl
Real part
Done DA
Gaussian part
Done Gaussian
Gaussian loss but with 'Real' DA values
Finished processing.
Loading file:  henon_eps_4.pkl
Real part
Done DA
Gaussian part
Done Gaussian
Gaussian loss but with 'Real' DA values
Finished processing.
Loading file:  henon_eps_5.pkl
Real part
Done DA
Gaussian part
Done Gaussian
Gaussian loss but with 'Real' DA values
Finished processing.
Loading file:  henon_eps_6.pkl
Real part
Done DA
Gaussian part
Done Gaussian
Gaussian loss but with 'Real' DA values
Finished processing.
Loading file:  henon_eps_7.pkl
Real part
Done DA
Gaussian part
Done Gaussian
Gaussian loss but with 'Real' DA values
Finished processing.
Loading file:  henon_eps_8.pkl
Rea

### Save data (since it takes a long time to generate)

In [5]:
with open("../data/henon_loss_data.pkl", "wb") as f:
    pickle.dump(
        {
            "sigma": sigma,
            "turn_samples": turn_samples,
            "min_turns": min_turns,
            "max_turns": max_turns,
            "epsilons": epsilons,
            "real_losses": real_losses,
            "gaussian_losses": gaussian_losses,
            "real_DAs": real_DAs,
            "gaussian_DAs": gaussian_DAs
        },
        f
    )

### Load data (if it was already computed before)

In [6]:
with open("../data/henon_loss_data.pkl", "rb") as f:
    dictionary = pickle.load(f)
    sigma = dictionary["sigma"]
    turn_samples = dictionary["turn_samples"]
    min_turns = dictionary["min_turns"]
    max_turns = dictionary["max_turns"]
    epsilons = dictionary["epsilons"]
    real_losses = dictionary["real_losses"]
    gaussian_losses = dictionary["gaussian_losses"]
    real_DAs = dictionary["real_DAs"]
    gaussian_DAs = dictionary["gaussian_DAs"]

## Visualize the various DA and Loss plots

In [7]:
plt.figure()

for i in range(len(real_losses)):
    plt.plot(turn_sampling, real_losses[i], c="C0")
    plt.plot(turn_sampling, gaussian_losses[i], c="C1")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
plt.figure()

for i in range(len(real_losses)):
    plt.plot(turn_sampling, real_DAs[i], c="C0")
    plt.plot(turn_sampling, gaussian_DAs[i], c="C1")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## General fitting of all the Hénon maps across all models

In [9]:
labels = (
    ("epsilon", ""),
    ("type", ""),
    ("sigma", ""),
    ("Model 2", "k"),
    ("Model 2", "k err"),
    ("Model 2", "rho"),
    ("Model 2", "rho err"),
    ("Model 2", "N0"),
    ("Model 2", "N0 err"),
    ("Model 2", "Chi2"),
    ("Model 4 (2 free pars)", "k"),
    ("Model 4 (2 free pars)", "k err"),
    ("Model 4 (2 free pars)", "rho"),
    ("Model 4 (2 free pars)", "rho err"),
    ("Model 4 (2 free pars)", "lambda"),
    ("Model 4 (2 free pars)", "Chi2"),
    ("Model 4 (3 free pars)", "k"),
    ("Model 4 (3 free pars)", "k err"),
    ("Model 4 (3 free pars)", "rho"),
    ("Model 4 (3 free pars)", "rho err"),
    ("Model 4 (3 free pars)", "N0"),
    ("Model 4 (3 free pars)", "N0 err"),
    ("Model 4 (3 free pars)", "lambda"),
    ("Model 4 (3 free pars)", "Chi2"),
)

fitting_data = pd.DataFrame(columns=pd.MultiIndex.from_tuples(labels))
lost_table = []

In [10]:
k_min = 0.1
k_max = 2.00
samples = 100

ks = np.linspace(k_min, k_max, samples)

In [11]:
for i, eps in tqdm(list(enumerate(epsilons))):
    # Model 2
    real_pars, real_errs, real_co_pars = fit.explore_k_model_2(
        turn_sampling, real_DAs[i], k_min, k_max, samples
    )
    gaussian_loss_pars, gaussian_loss_errs, gaussian_loss_co_pars = fit.explore_k_model_2(
        turn_sampling, gaussian_DAs[i], k_min, k_max, samples
    )

    real_selected_err_2 = np.min(real_errs)
    real_selected_k_2 = ks[np.argmin(real_errs)]
    real_selected_pars_2 = real_pars[np.argmin(real_errs)]
    real_selected_co_pars_2 = real_co_pars[np.argmin(real_errs)]

    gaussian_loss_selected_err_2 = np.min(gaussian_loss_errs)
    gaussian_loss_selected_k_2 = ks[np.argmin(gaussian_loss_errs)]
    gaussian_loss_selected_pars_2 = gaussian_loss_pars[np.argmin(gaussian_loss_errs)]
    gaussian_loss_selected_co_pars_2 = gaussian_loss_co_pars[np.argmin(gaussian_loss_errs)]
    
    # Model 4 (2 pars)
    real_pars, real_errs, real_co_pars = fit.explore_k_model_4(
        turn_sampling, real_DAs[i], k_min, k_max, samples
    )
    gaussian_loss_pars, gaussian_loss_errs, gaussian_loss_co_pars = fit.explore_k_model_4(
        turn_sampling, gaussian_DAs[i], k_min, k_max, samples
    )

    real_selected_err_4 = np.min(real_errs)
    real_selected_k_4 = ks[np.argmin(real_errs)]
    real_selected_pars_4 = real_pars[np.argmin(real_errs)]
    real_selected_co_pars_4 = real_co_pars[np.argmin(real_errs)]

    gaussian_loss_selected_err_4 = np.min(gaussian_loss_errs)
    gaussian_loss_selected_k_4 = ks[np.argmin(gaussian_loss_errs)]
    gaussian_loss_selected_pars_4 = gaussian_loss_pars[np.argmin(gaussian_loss_errs)]
    gaussian_loss_selected_co_pars_4 = gaussian_loss_co_pars[np.argmin(gaussian_loss_errs)]
    
    # Model 4 (3 pars)
    real_pars, real_errs, real_co_pars = fit.explore_model_4_free(
        turn_sampling, real_DAs[i], k_min, k_max, samples
    )
    gaussian_loss_pars, gaussian_loss_errs, gaussian_loss_co_pars = fit.explore_model_4_free(
        turn_sampling, gaussian_DAs[i], k_min, k_max, samples
    )

    real_selected_err_4_free = np.min(real_errs)
    real_selected_k_4_free = ks[np.argmin(real_errs)]
    real_selected_pars_4_free = real_pars[np.argmin(real_errs)]
    real_selected_co_pars_4_free = real_co_pars[np.argmin(real_errs)]

    gaussian_loss_selected_err_4_free = np.min(gaussian_loss_errs)
    gaussian_loss_selected_k_4_free = ks[np.argmin(gaussian_loss_errs)]
    gaussian_loss_selected_pars_4_free = gaussian_loss_pars[np.argmin(gaussian_loss_errs)]
    gaussian_loss_selected_co_pars_4_free = gaussian_loss_co_pars[np.argmin(gaussian_loss_errs)]
    
    # Saving Data
    fitting_data.loc[len(fitting_data)] = [
        epsilons[i],
        "real",
        np.nan,
        real_selected_k_2,
        ks[1] - ks[0],
        real_selected_pars_2[0],
        np.sqrt(real_selected_co_pars_2[0][0]),
        real_selected_pars_2[1],
        np.sqrt(real_selected_co_pars_2[1][1]),
        real_selected_err_2,
        real_selected_k_4,
        ks[1] - ks[0],
        real_selected_pars_4[0],
        np.sqrt(real_selected_co_pars_4[0][0]),
        1/2,
        real_selected_err_4,
        real_selected_k_4_free,
        ks[1] - ks[0],
        real_selected_pars_4_free[0],
        np.sqrt(real_selected_co_pars_4_free[0][0]),
        real_selected_pars_4_free[1],
        np.sqrt(real_selected_co_pars_4_free[1][1]),
        1/2,
        real_selected_err_4_free
    ]
    
    lost_table.append(real_losses[i])
    
    fitting_data.loc[len(fitting_data)] = [
        epsilons[i],
        "gaussian",
        sigma,
        gaussian_loss_selected_k_2,
        ks[1] - ks[0],
        gaussian_loss_selected_pars_2[0],
        np.sqrt(gaussian_loss_selected_co_pars_2[0][0]),
        gaussian_loss_selected_pars_2[1],
        np.sqrt(gaussian_loss_selected_co_pars_2[1][1]),
        gaussian_loss_selected_err_2,
        gaussian_loss_selected_k_4,
        ks[1] - ks[0],
        gaussian_loss_selected_pars_4[0],
        np.sqrt(gaussian_loss_selected_co_pars_4[0][0]),
        1/2,
        gaussian_loss_selected_err_4,
        gaussian_loss_selected_k_4_free,
        ks[1] - ks[0],
        gaussian_loss_selected_pars_4_free[0],
        np.sqrt(gaussian_loss_selected_co_pars_4_free[0][0]),
        gaussian_loss_selected_pars_4_free[1],
        np.sqrt(gaussian_loss_selected_co_pars_4_free[1][1]),
        1/2,
        gaussian_loss_selected_err_4_free
    ]
    
    lost_table.append(gaussian_losses[i])

### Visualize the $\kappa$ value for the various models

In [12]:
fig1, ax1 = plt.subplots(1,2)

p_filter = fitting_data[("type", "")] == "real" 

ax1[0].scatter(
    fitting_data[p_filter][("epsilon", "")],
    fitting_data[p_filter][("Model 2", "k")],
    label="Model 2"
)
ax1[0].scatter(
    fitting_data[p_filter][("epsilon", "")],
    fitting_data[p_filter][("Model 4 (2 free pars)", "k")],
    label="Model 4 (2 pars)"
)
ax1[0].scatter(
    fitting_data[p_filter][("epsilon", "")],
    fitting_data[p_filter][("Model 4 (3 free pars)", "k")],
    label="Model 4 (3 pars)"
)

ax1[0].set_xlabel("$\\epsilon$")
ax1[0].set_ylabel("$\\kappa$")
ax1[0].set_title("Real DA")
ax1[0].legend(fontsize="xx-small")

p_filter = fitting_data[("type", "")] == "gaussian" 

ax1[1].scatter(
    fitting_data[p_filter][("epsilon", "")],
    fitting_data[p_filter][("Model 2", "k")],
    label="Model 2"
)
ax1[1].scatter(
    fitting_data[p_filter][("epsilon", "")],
    fitting_data[p_filter][("Model 4 (2 free pars)", "k")],
    label="Model 4 (2 pars)"
)
ax1[1].scatter(
    fitting_data[p_filter][("epsilon", "")],
    fitting_data[p_filter][("Model 4 (3 free pars)", "k")],
    label="Model 4 (3 pars)"
)

ax1[1].set_xlabel("$\\epsilon$")
ax1[1].set_ylabel("$\\kappa$")
ax1[1].set_title("DA from Gaussian loss")
ax1[1].legend(fontsize="xx-small")

p_filter = fitting_data[("type", "")] == "uniform" 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
fitting_data[["epsilon", "type", "sigma", "Model 2"]]

epsilon      type sigma   Model 2                                            \
                                  k     k err       rho   rho err          N0   
0     2.0      real   NaN  0.234343  0.019192  1.832158  0.001785    3.605546   
1     2.0  gaussian   0.3  0.157576  0.019192  1.207313  0.000374  121.248639   
2     4.0      real   NaN  0.349495  0.019192  3.256393  0.003599    0.159521   
3     4.0  gaussian   0.3  0.215152  0.019192  1.597565  0.001023   33.866180   
4     6.0      real   NaN  0.387879  0.019192  3.867082  0.002893    0.095018   
5     6.0  gaussian   0.3  0.253535  0.019192  1.912311  0.001438   16.358605   
6     8.0      real   NaN  0.407071  0.019192  4.186000  0.002269    0.083867   
7     8.0  gaussian   0.3  0.291919  0.019192  2.290799  0.001684    7.581011   
8    10.0      real   NaN  0.541414  0.019192  8.105351  0.007333    0.002843   
9    10.0  gaussian   0.3  0.330303  0.019192  2.744852  0.002012    3.457098   

                       
     N0 err      Chi2  
0  0.167194  0.000098  
1  1.698826  0.000051  
2  0.007243  0.000061  
3  0.875988  0.000103  
4  0.002727  0.000025  
5  0.460542  0.000105  
6  0.001676  0.000013  
7  0.196987  0.000076  
8  0.000088  0.000022  
9  0.085502  0.000058

In [14]:
fitting_data[["epsilon", "type", "sigma", "Model 4 (2 free pars)"]]

epsilon      type sigma Model 4 (2 free pars)                                \
                                              k     k err       rho   rho err   
0     2.0      real   NaN              0.291919  0.019192  2.519587  0.000444   
1     2.0  gaussian   0.3              0.330303  0.019192  3.018065  0.001425   
2     4.0      real   NaN              0.330303  0.019192  2.958804  0.000345   
3     4.0  gaussian   0.3              0.368687  0.019192  3.542396  0.001233   
4     6.0      real   NaN              0.349495  0.019192  3.190777  0.000289   
5     6.0  gaussian   0.3              0.387879  0.019192  3.821020  0.000980   
6     8.0      real   NaN              0.368687  0.019192  3.444867  0.000257   
7     8.0  gaussian   0.3              0.407071  0.019192  4.122350  0.000853   
8    10.0      real   NaN              0.387879  0.019192  3.713706  0.000526   
9    10.0  gaussian   0.3              0.407071  0.019192  4.101843  0.000766   

                    
  lambda      Chi2  
0    0.5  0.000139  
1    0.5  0.000997  
2    0.5  0.000064  
3    0.5  0.000553  
4    0.5  0.000039  
5    0.5  0.000292  
6    0.5  0.000025  
7    0.5  0.000192  
8    0.5  0.000093  
9    0.5  0.000142

In [15]:
fitting_data[["epsilon", "type", "sigma", "Model 4 (3 free pars)"]]

epsilon      type sigma Model 4 (3 free pars)                                \
                                              k     k err       rho   rho err   
0     2.0      real   NaN              0.234343  0.019192  1.837705  0.001754   
1     2.0  gaussian   0.3              0.157576  0.019192  1.210471  0.000353   
2     4.0      real   NaN              0.349495  0.019192  3.272338  0.003550   
3     4.0  gaussian   0.3              0.215152  0.019192  1.603340  0.000980   
4     6.0      real   NaN              0.387879  0.019192  3.889447  0.002858   
5     6.0  gaussian   0.3              0.272727  0.019192  2.118096  0.001726   
6     8.0      real   NaN              0.426263  0.019192  4.653504  0.002634   
7     8.0  gaussian   0.3              0.311111  0.019192  2.537796  0.001998   
8    10.0      real   NaN              0.503030  0.019192  6.713119  0.005862   
9    10.0  gaussian   0.3              0.330303  0.019192  2.761387  0.001953   

                                          
           N0    N0 err lambda      Chi2  
0    4.138362  0.188347    0.5  0.000098  
1  125.077652  1.655799    0.5  0.000048  
2    0.207906  0.009280    0.5  0.000061  
3   36.832575  0.911685    0.5  0.000099  
4    0.128593  0.003631    0.5  0.000025  
5   10.981722  0.331730    0.5  0.000104  
6    0.064032  0.001331    0.5  0.000013  
7    5.341587  0.147444    0.5  0.000075  
8    0.014236  0.000427    0.5  0.000025  
9    4.199171  0.100511    0.5  0.000057

# Fokker-Planck Fittings
## Fixed $\kappa$, exploring $I_\ast$

In [16]:
labels = (
    ("epsilon", ""),
    ("type", ""),
    ("sigma", ""),
    ("Fokker-Planck", "I_star"),
    ("Fokker-Planck", "time_scale"),
    ("Fit data", "error"),
)

fp_data = []
fp_fit_data = pd.DataFrame(columns=pd.MultiIndex.from_tuples(labels))

In [17]:
for i in tqdm(range(len(fitting_data[("Model 4 (3 free pars)", "k")]))):
    # Setup initial conditions
    I_max = cut_point**2 / 2
    I = np.linspace(0, I_max, 1000)
    I0 = I * np.exp(-(I/sigma**2))
    I0 /= integrate.trapz(I0, I)
    # Execute research
    values, I_star, t, error = fit.scan_I_star(
        lost_table[i],
        turn_sampling,
        I_max,
        I0,
        fitting_data[("Model 4 (3 free pars)", "k")][i],
        0.2,
        0.01,
        10,
        0.05
    )
    # Save data
    fp_data.append(values)
    fp_fit_data.loc[len(fp_fit_data)] = [
        fitting_data[("epsilon", "")][i],
        fitting_data[("type", "")][i],
        fitting_data[("sigma", "")][i],
        I_star,
        t,
        error
    ]

k=0.23434343434343433, I_star=0.19, dt=0.01
k=0.23434343434343433, I_star=0.2, dt=0.01
k=0.23434343434343433, I_star=0.21000000000000002, dt=0.01
Going UP!
k=0.23434343434343433, I_star=0.22050000000000003, dt=0.01
k=0.23434343434343433, I_star=0.23152500000000004, dt=0.01
k=0.23434343434343433, I_star=0.24310125000000005, dt=0.01
k=0.23434343434343433, I_star=0.2552563125000001, dt=0.01
k=0.23434343434343433, I_star=0.2680191281250001, dt=0.01
k=0.23434343434343433, I_star=0.2814200845312501, dt=0.01
k=0.23434343434343433, I_star=0.29549108875781266, dt=0.01
increase!
k=0.23434343434343433, I_star=0.3102656431957033, dt=0.1
k=0.23434343434343433, I_star=0.3257789253554885, dt=0.1
k=0.1575757575757576, I_star=0.19, dt=0.01
k=0.1575757575757576, I_star=0.2, dt=0.01
k=0.1575757575757576, I_star=0.21000000000000002, dt=0.01
Going UP!
k=0.1575757575757576, I_star=0.22050000000000003, dt=0.01
k=0.1575757575757576, I_star=0.23152500000000004, dt=0.01
k=0.3494949494949495, I_star=0.19, dt=0.0

In [18]:
fp_fit_data

epsilon      type sigma Fokker-Planck  \
                                 I_star   
0     2.0      real   NaN      0.310266   
1     2.0  gaussian   0.3      0.220500   
2     4.0      real   NaN      0.530660   
3     4.0  gaussian   0.3      0.268019   
4     6.0      real   NaN      0.645020   
5     6.0  gaussian   0.3      0.342068   
6     8.0      real   NaN      0.784026   
7     8.0  gaussian   0.3      0.395986   
8    10.0      real   NaN      1.158363   
9    10.0  gaussian   0.3      0.436575   

                                                      Fit data  
                                          time_scale     error  
0  [500.0, 499.0, 498.0, 497.0, 496.0, 495.0, 494...  0.008507  
1  [50.2, 50.1, 50.0, 49.9, 49.800000000000004, 4...  0.020607  
2  [7110.0, 7100.0, 7090.0, 7080.0, 7070.0, 7060....  0.007976  
3  [79.0, 78.9, 78.8, 78.7, 78.60000000000001, 78...  0.006507  
4  [23520.0, 23510.0, 23500.0, 23490.0, 23480.0, ...  0.009448  
5  [299.0, 298.0, 297.0, 296.0, 295.0, 294.0, 293...  0.010073  
6  [60900.0, 60800.0, 60700.0, 60600.0, 60500.0, ...  0.010069  
7  [435.0, 434.0, 433.0, 432.0, 431.0, 430.0, 429...  0.010699  
8  [232900.0, 232800.0, 232700.0, 232600.0, 23250...  0.009763  
9  [910.0, 909.0, 908.0, 907.0, 906.0, 905.0, 904...  0.011310

In [19]:
plt.figure()

plt.plot(
    fp_fit_data[fp_fit_data[("type")] == "gaussian"][("epsilon")],
    fp_fit_data[fp_fit_data[("type")] == "gaussian"][("Fokker-Planck", "I_star")]
)
plt.xlabel("$\\varepsilon$")
plt.ylabel("$I_\\ast$")
plt.title("Best value for $I_\\ast$")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Best value for $I_\\ast$')

## Double FP fit for both $\kappa$ and $I_\ast$

In [20]:
labels = (
    ("epsilon", ""),
    ("type", ""),
    ("sigma", ""),
    ("Fokker-Planck", "k"),
    ("Fokker-Planck", "I_star"),
    ("Fokker-Planck", "time_scale"),
    ("Fit data", "error")
)

fp_double_data = []
fp_double_fit_data = pd.DataFrame(columns=pd.MultiIndex.from_tuples(labels))

In [21]:
for i in tqdm(range(len(fitting_data[("Model 4 (3 free pars)", "k")]))):
    # Setup initial conditions
    I_max = cut_point**2 / 2
    I = np.linspace(0, I_max, 1000)
    I0 = I * np.exp(-(I/sigma**2))
    I0 /= integrate.trapz(I0, I)
    # Execute research
    values, k, I_star, t, error = fit.scan_k(
        lost_table[i],
        turn_sampling,
        I_max,
        I0,
        fitting_data[("Model 4 (3 free pars)", "k")][i],
        0.2,
        0.001,
        10,
        0.05,
        0.05
    )
    fp_double_data.append(values)
    fp_double_fit_data.loc[len(fp_double_fit_data)] = [
        fitting_data[("epsilon", "")][i],
        fitting_data[("type", "")][i],
        fitting_data[("sigma", "")][i],
        k,
        I_star,
        t,
        error,
    ]

k=0.2226262626262626, I_star=0.19, dt=0.001
k=0.2226262626262626, I_star=0.2, dt=0.001
k=0.2226262626262626, I_star=0.21000000000000002, dt=0.001
Going UP!
k=0.2226262626262626, I_star=0.22050000000000003, dt=0.001
k=0.2226262626262626, I_star=0.23152500000000004, dt=0.001
k=0.2226262626262626, I_star=0.24310125000000005, dt=0.001
k=0.2226262626262626, I_star=0.2552563125000001, dt=0.001
increase!
k=0.2226262626262626, I_star=0.2680191281250001, dt=0.01
k=0.2226262626262626, I_star=0.2814200845312501, dt=0.01
increase!
k=0.2226262626262626, I_star=0.29549108875781266, dt=0.1
k=0.23434343434343433, I_star=0.19, dt=0.001
k=0.23434343434343433, I_star=0.2, dt=0.001
k=0.23434343434343433, I_star=0.21000000000000002, dt=0.001
Going UP!
k=0.23434343434343433, I_star=0.22050000000000003, dt=0.001
k=0.23434343434343433, I_star=0.23152500000000004, dt=0.001
k=0.23434343434343433, I_star=0.24310125000000005, dt=0.001
k=0.23434343434343433, I_star=0.2552563125000001, dt=0.001
k=0.2343434343434343

In [22]:
i = 4

plt.figure()
plt.plot(turn_sampling, fp_double_data[i][:-1])
plt.plot(turn_sampling, lost_table[i])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
fp_double_fit_data

epsilon      type sigma Fokker-Planck            \
                                      k    I_star   
0     2.0      real   NaN      0.211495  0.268019   
1     2.0  gaussian   0.3      0.149697  0.210000   
2     4.0      real   NaN      0.299648  0.395986   
3     4.0  gaussian   0.3      0.194174  0.243101   
4     6.0      real   NaN      0.315930  0.415786   
5     6.0  gaussian   0.3      0.286364  0.359171   
6     8.0      real   NaN      0.404949  0.677271   
7     8.0  gaussian   0.3      0.311111  0.395986   
8    10.0      real   NaN      0.389235  0.585052   
9    10.0  gaussian   0.3      0.313788  0.395986   

                                                      Fit data  
                                          time_scale     error  
0  [87.9, 87.8, 87.7, 87.60000000000001, 87.5, 87...  0.005022  
1  [23.18, 23.17, 23.16, 23.150000000000002, 23.1...  0.004462  
2  [861.0, 860.0, 859.0, 858.0, 857.0, 856.0, 855...  0.006829  
3  [40.300000000000004, 40.2, 40.1, 40.0, 39.9, 3...  0.004988  
4  [1000.0, 999.0, 998.0, 997.0, 996.0, 995.0, 99...  0.006302  
5  [291.0, 290.0, 289.0, 288.0, 287.0, 286.0, 285...  0.010038  
6  [19590.0, 19580.0, 19570.0, 19560.0, 19550.0, ...  0.009202  
7  [435.0, 434.0, 433.0, 432.0, 431.0, 430.0, 429...  0.010699  
8  [5700.0, 5690.0, 5680.0, 5670.0, 5660.0, 5650....  0.007229  
9  [447.0, 446.0, 445.0, 444.0, 443.0, 442.0, 441...  0.009293

In [24]:
plt.figure()
plt.scatter(
    fp_double_fit_data[fp_double_fit_data["type"] == "gaussian"]["epsilon"],
    fp_double_fit_data[fp_double_fit_data["type"] == "gaussian"][("Fokker-Planck", "k")],
    label="$\\kappa$, FP best fit"
)
plt.scatter(
    fitting_data[fitting_data["type"] == "gaussian"]["epsilon"],
    fitting_data[fitting_data["type"] == "gaussian"][("Model 4 (3 free pars)", "k")],
    label="$\\kappa$, Model 4 best fit"
)

plt.legend()
plt.xlabel("$\\varepsilon$")
plt.ylabel("$\\kappa$")
plt.title("Comparison between Model 4 best fit and Fokker-Planck result")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Comparison between Model 4 best fit and Fokker-Planck result')

In [25]:
plt.figure()
plt.scatter(
    fp_double_fit_data[fp_double_fit_data["type"] == "gaussian"]["epsilon"],
    fp_double_fit_data[fp_double_fit_data["type"] == "gaussian"][("Fokker-Planck", "I_star")],
    label="$I_\\ast$, FP best fit"
)

plt.legend()
plt.xlabel("$\\varepsilon$")
plt.ylabel("$I_\\ast$")
plt.title("Comparison between Model 4 best fit and Fokker-Planck result")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Comparison between Model 4 best fit and Fokker-Planck result')